In [1]:
%gui qt
import os
import sys
import re
import StringFinderWin
import StringFormatterWin
from datetime import datetime
from abc import ABC, abstractmethod
from PyQt5.QtWidgets import QMainWindow, QWidget, QApplication, QListView, QMenuBar, QLabel, QFileDialog, QMessageBox
from PyQt5.QtCore import QStringListModel, QCoreApplication
from enum import Enum, unique

1. Задан простой класс Fraction для представления дробей:
class Fraction(object):
    def __init__(self, num, den):
        self.__num = num
        self.__den = den
        self.reduce()
    def __str__(self):
        return "%d/%d" % (self.__num, self.__den)
    def reduce(self):
        g = Fraction.gcd(self.__num, self.__den)
        self.__num /= g
        self.__den /= g
    @staticmethod
    def gcd(n, m):
        if m == 0:
            return n
        else:
            return Fraction.gcd(m, n % m)
Дополнить класс таким образом, чтобы выполнялся следующий код:

frac = Fraction(7, 2)

print(-frac) # выводит -7/2

print(~frac) # выводит 2/7

print(frac**2) # выводит 49/4

print(float(frac)) # выводит 3.5

print(int(frac)) # выводит 3

In [3]:
class Fraction(object):
    def __init__(self, num, den):
        self.__num = num
        self.__den = den
        self.reduce()
    def __str__(self):
        return "%d/%d" % (self.__num, self.__den)
    def reduce(self):
        g = Fraction.gcd(self.__num, self.__den)
        self.__num /= g
        self.__den /= g
    @staticmethod
    def gcd(n, m):
        if m == 0:
            return n
        else:
            return Fraction.gcd(m, n % m)
        
    #Пользовательские "магические" методы:
    #Приведения типов:
    def __int__(self):
        return int(self.__num // self.__den)

    def __float__(self):
        return self.__num / self.__den
    
    #Унарные операции преобразования в отрицательное и инвертирования (~):
    def __neg__(self):
        return Fraction(-self.__num, self.__den)

    def __invert__(self):
        return Fraction(self.__den, self.__num)

    #Бинарная операция возведения в степень:
    def __pow__(self, power, modulo=None):
        return Fraction(self.__num ** power, self.__den ** power)

#Запросы по заданию:
frac = Fraction(7, 2)
print(-frac)
print(~frac)
print(frac**2)
print(float(frac))
print(int(frac))

-7/2
2/7
49/4
3.5
3


2. Напишите классы «Книга» (с обязательными полями: название, автор,
код), «Библиотека» (с обязательными полями: адрес, номер) и
корректно свяжите их. Код книги должен назначаться автоматически
при добавлении книги в библиотеку (используйте для этого
статический член класса). Если в конструкторе книги указывается в
параметре пустое название, необходимо сгенерировать исключение
(например, ValueError). Книга должна реализовывать интерфейс
Taggable с методом tag(), который создает на основе строки набор тегов
(разбивает строку на слова и возвращает только те, которые
начинаются с большой буквы). Например, tag() для книги с названием
‘War and Peace’ вернет список тегов [‘War’, ‘Peace’]. Реализуйте классы
таким образом, чтобы корректно выполнялся следующий код:
lib = Library(1, ’51 Some str., NY’)
lib += Book(‘Leo Tolstoi’, ‘War and Peace’)
lib += Book(‘Charles Dickens’, ‘David Copperfield’)
for book in lib:
#Вывод в виде [1] L.Tolstoi 'War and Peace'
print(book)
#Вывод в виде ['War', 'Peace']
print(book.tag())

In [3]:
class Taggable(ABC):
    #Метод tag, который необходимо реализовать во всех реализациях абстрактного класса
    @abstractmethod
    def tag(self):
        pass
    
class Book(Taggable):    
    def __init__(self, author: str, name: str):
        if author and name:
            self.__name = name
            self.__author = author
            self.__code = None
        else:
            raise ValueError('Empty name.')
    
    def tag(self):
        return list(filter(lambda x: x[0].isupper(), self.__name.split()))
    
    def __repr__(self):
        code_string = ''
        if self.__code:
            code_string = '[{}]'.format(self.__code)
        return "{} {}.{} '{}'".format(
                code_string, self.__author[0], self.__author.split()[1], self.__name)
    
    def set_code(self, code_):
        self.__code = code_
        
class Library(object):
    __current_code = 0
    
    def __init__(self, number, address):
        self.__number = number
        self.__address = address
        self.__books = list()
    
    def __iadd__(self, book: Book):
        Library.__current_code += 1
        book.set_code(Library.__current_code)
        self.__books.append(book)
        return self
    
    def __iter__(self):
        for book in self.__books:
            yield book

#Запросы по заданию:
lib = Library(1, '51 Some str., NY')
lib += Book('Leo Tolstoi', 'War and Peace')
lib += Book('Charles Dickens', 'David Copperfield')
for book in lib:
    # вывод в виде: [1] L.Tolstoi ‘War and Peace’
    print(book)
    # вывод в виде: [‘War’, ‘Peace’]
    print(book.tag())

[1] L.Tolstoi 'War and Peace'
['War', 'Peace']
[2] C.Dickens 'David Copperfield'
['David', 'Copperfield']


3.Создайте графическую оболочку для скрипта, написанного в ходе
выполнения задания № 4 лабораторной работы № 2, в виде диалогового
окна. Рекомендуется использовать wxPython или PyQt.
Требования к окну и скрипту:
- всю область окна должен занимать список с результатами поиска
строк по шаблону в файле и указанием даты и времени поиска.
Поиск производится автоматически при каждом открытии какоголибо файла, при этом список не очищается, а пополняется новыми
результатами. При запуске скрипта список изначально должен быть
пустым (из файла лога данные подгружать не нужно);
- строка меню содержит пункты «Файл» (с подпунктом «Открыть...»
для открытия файла, в котором необходимо искать строки) и «Лог»
(с подпунктами «Экспорт...», «Добавить в лог», «Просмотр»). Файл
лога находится в рабочей папке скрипта и называется script18.log.
Если файл отсутствует, скрипт при запуске должен выдать
диалоговое окно с информацией «Файл лога не найден. Файл будет
создан автоматически» и кнопкой «ОК». При выборе пункта меню
«Экспорт...» содержимое списка должно сохраниться в файле,
который укажет пользователь. При выборе пункта «Добавить в лог»
содержимое списка приписывается в конец файла script18.log. При
выборе пункта «Просмотр» текущее содержимое списка удаляется,
и список заполняется данными из лога. Перед этим действием скрипт
должен выдать диалоговое окно с вопросом «Вы действительно
хотите открыть лог? Данные последних поисков будут потеряны!»
и кнопками «Да» и «Нет»;
- статусная строка должна состоять из двух полей: в первом поле (60%
ширины окна), в зависимости от последнего произведенного
действия, выводится либо текст «Открыт лог», либо текст
«Обработан файл <полное_имя_файла>»; второе поле (40%
ширины окна) служит для отображения размера последнего
обработанного файла в байтах. Эта строка форматируется: выводятся
пробелы между степенями тысячи (например, «2 036 231 байт»);
- файлы нужно открывать и сохранять с помощью стандартного
диалогового окна.

In [ ]:
class StringFinderWindow(QMainWindow):
    def __init__(self):
        #Импорт UI окна
        QMainWindow.__init__(self)
        self.ui = StringFinderWin.Ui_MainWindow()
        self.ui.setupUi(self)
        self.list_logs_model = QStringListModel()
        self.ui.list_logs.setModel(self.list_logs_model)
        #Привязка действий к кнопкам меню
        self.ui.actionOpen.triggered.connect(self.trigger_action_open)
        self.ui.actionExport.triggered.connect(self.trigger_action_export)
        self.ui.actionAdd.triggered.connect(self.trigger_action_add)
        self.ui.actionView.triggered.connect(self.trigger_action_view)
        #Настройка Status bar
        self.status = QLabel('')
        self.statusBar().addWidget(self.status, self.width() * 0.6)
        self.last_size_status = QLabel('')
        self.statusBar().addPermanentWidget(self.last_size_status, self.width() * 0.4)
        self.log = "log_task3.txt"
        self.log_creation()
    
    def trigger_action_open(self):
        filename = QFileDialog().getOpenFileName(self, 'Open file', sys.path[0])[0]
        if filename:
            self.find_string(filename)
            self.status.setText('Файл ' + filename)
            self.last_size_status.setText(self.file_size(filename))
    
    def trigger_action_export(self):
        filename = QFileDialog().getSaveFileName(self, 'Save file', sys.path[0])[0]
        if filename:
            with open(filename, 'w') as f:
                for sl in self.list_logs_model.stringList():
                    f.write(sl + '\n')
    
    def trigger_action_add(self):
        with open(self.log, 'a') as f:
            for sl in self.list_logs_model.stringList():
                f.write(sl + '\n')
    
    def trigger_action_view(self):
        text_msg = 'Открыть лог? Несохраненные данные будут утеряны.'
        msg = QMessageBox.question(self, 'Внимание', text_msg, buttons=QMessageBox.Yes | QMessageBox.No)
        if msg == QMessageBox.Yes:
            with open(self.log, 'r') as f:
                self.list_logs_model.setStringList(f.read().split('\n'))
            self.status.setText('Открыт лог')
    
    def find_string(self, filename):
        try:
            with open(filename, 'r', encoding='utf8') as f:
                text = f.read().split('\n')
        except Exception as e:
            finder = ['Файл ' + filename, ' ', 'Не удалось прочитать файл..', ' ']
        if text:
            date_time = datetime.now().strftime("%d/%m/%Y, %H:%M:%S")
            finder = ['Файл ' + filename + ' был обработан ' + date_time, ' '] + ["Строка {}, позиция {} : найдено '{}'".format(j, m.start(), m.group(0))
                                           for j, t in enumerate(text) for m in re.finditer(r"(([0-1]\d|[2][0-3]):([0-5]\d):([0-5]\d))", t)]
            if finder[-1] == '':
                find.append('Ничего не найдено.')
            finder.append('')
        self.list_logs_model.setStringList(self.list_logs_model.stringList() + finder)
        
    def log_creation(self):
        if not os.path.exists(os.path.abspath(self.log)):
            log_text = 'Файл лога не найден. Файл будет создан автоматически'
            QMessageBox.warning(self, 'Внимание', log_text)
            with open(self.log, 'w'):
                pass
    
    @staticmethod
    def file_size(file):
        size = str(os.path.getsize(file))
        l = len(size)
        sl = [size[l - 3 * i - 3:l - 3 * i] for i in range(l // 3, -1, -1)]
        return size[0: l % 3] + ' '.join(sl) + ' байт'

if __name__ == '__main__':
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    w = StringFinderWindow()
    w.show()

    app.exec_()

4. Напишите простой класс StringFormatter для форматирования строк со
следующим функционалом:
– удаление всех слов из строки, длина которых меньше n букв;
– замена всех цифр в строке на знак «*»;
– вставка по одному пробелу между всеми символами в строке;
– сортировка слов по размеру;
– сортировка слов в лексикографическом порядке.
Примечание. Разделители слов можно задавать отдельно. По
умолчанию в качестве разделителя принимается только символ
пробела.

In [3]:
class StringFormatter(object):
    def __init__(self, min_length=None, replace=False, divide_by_space=False, sort="NoSort"):
        self.__min_length = min_length
        self.__divide_by_space = divide_by_space
        self.__sort = sort
        self.__replace = replace
        pass

    #Сеттеры
    def set_sort(self, sort):
        self.__sort = sort

    def set_replace(self, rep: bool):
        self.__replace = rep

    def set_min_length(self, len):
        self.__min_length = len

    def set_space(self, divide_by_space):
        self.__divide_by_space = divide_by_space

    #Методы форматирования
    def __filter__(self, string_list):
        return list(filter(lambda x: len(x) >= self.__min_length, string_list))

    def __replace__(self, string_list):
        return [i.translate(str.maketrans('0123456789', '*' * 10)) for i in string_list]

    def __space__(self, string_list):
        return [' '.join([s for s in sl]) for sl in string_list]

    def __sort__(self, string_list: list):
        if self.__sort == "ByLexic":
            return sorted(string_list, key=str.lower)
        if self.__sort == "ByLength":
            return sorted(string_list, key=len)

    def format(self, string: str) -> str:
        text = string.split()
        if self.__min_length:
            text = self.__filter__(text)
        if self.__replace:
            text = self.__replace__(text)
        if self.__divide_by_space:
            text = self.__space__(text)
        if self.__sort != "NoSort":
            text = self.__sort__(text)
        return ' '.join(text)

formatter = StringFormatter(min_length=5, sort="ByLength")
test_string = 'alt alternative alte alternativ alter alternati altern alternat alterna'
print(formatter.format(test_string))

alter altern alterna alternat alternati alternativ alternative


5. Напишите скрипт с графическим интерфейсом пользователя для
демонстрации работы класса StringFormatter. Примеры окон приведены
на рис. 4 (все элементы управления необходимо обязательно
реализовать те же, что присутствуют на рисунке). Разные комбинации
отмеченных чекбоксов приводят к разным цепочкам операций
форматирования задаваемой в верхнем поле строки с разными
результатами.

In [ ]:
class StringFormatterWindow(QWidget):
    def __init__(self):
        #Импорт UI окна
        QWidget.__init__(self)
        self.ui = StringFormatterWin.Ui_Form()
        self.ui.setupUi(self)
        #Настройка изначального состояния чекбоксом и радиокнопок, привязка действий
        self.ui.spinBox_HowMuch.setEnabled(False)
        self.ui.radioButton_BySize.setEnabled(False)
        self.ui.radioButton_byLexic.setEnabled(False)
        self.ui.checkBox_Delete.clicked.connect(lambda x: self.ui.spinBox_HowMuch.setEnabled(
            not self.ui.spinBox_HowMuch.isEnabled()))
        self.ui.checkBox_Sort.clicked.connect(self.check_sort)
        self.ui.button_format.clicked.connect(self.__button_format)
        
    def check_sort(self):
        self.ui.radioButton_BySize.setChecked(False)
        self.ui.radioButton_BySize.setEnabled(not self.ui.radioButton_BySize.isEnabled())
        self.ui.radioButton_byLexic.setChecked(False)
        self.ui.radioButton_byLexic.setEnabled(not self.ui.radioButton_byLexic.isEnabled())
    
    def __button_format(self):
        formatter = StringFormatter()
        if self.ui.checkBox_Delete.isChecked():
            formatter.set_min_length(self.ui.spinBox_HowMuch.value())
        if self.ui.checkBox_Ast.isChecked():
            formatter.set_replace(True)
        if self.ui.checkBox_Spaces.isChecked():
            formatter.set_space(True)
        if self.ui.checkBox_Sort.isChecked():
            if self.ui.radioButton_BySize.isChecked():
                formatter.set_sort("ByLength")
            else:
                formatter.set_sort("ByLexic")
        self.ui.line_Result.setText(formatter.format(self.ui.line_Input.text()))
        
if __name__ == '__main__':
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    w = StringFormatterWindow()
    w.show()

    app.exec_()